# Introduction to Xoak

This notebook briefly shows how to use Xoak with Xarray's [advanced indexing](http://xarray.pydata.org/en/stable/indexing.html#more-advanced-indexing) to perform point-wise selection of irrelgularly spaced data encoded in coordinates with an arbitrary number of dimensions (1, 2, ..., n-d).

**Note**: Xoak relies on [xarray.indexes.NDPointIndex](https://docs.xarray.dev/en/stable/generated/xarray.indexes.NDPointIndex.html), which has been added in Xarray version 2025.07.1.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xoak

xr.set_options(display_expand_indexes=True);

Let's first create an `xarray.Dataset` of latitude / longitude points located randomly on the sphere, forming a 2-dimensional (x, y) model mesh (note that Xoak supports indexing coordinates with an arbitrary number of dimensions).

In [ ]:
shape = (100, 100)
lat = np.random.uniform(-90, 90, size=shape)
lon = np.random.uniform(-180, 180, size=shape)

field = lat + lon

In [ ]:
ds_mesh = xr.Dataset(
    coords={'lat': (('x', 'y'), lat), 'lon': (('x', 'y'), lon)},
    data_vars={'field': (('x', 'y'), field)},
)

ds_mesh

We first need to build an index to allow fast point-wise selection. Xoak extends [xarray.indexes.NDPointIndex](https://docs.xarray.dev/en/stable/generated/xarray.indexes.NDPointIndex.html) with different structures available for use cases depending on the data. In the example below we use a wrapper around [sklearn.BallTree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#sklearn.neighbors.BallTree) using the Haversine distance metric that is suited for indexing latitude / longitude points.

With this index, it is important to specify `lat` and `lon` in this specific order. Both the `lat` and `lon` coordinates must have exactly the same dimensions in the same order, here `('x', 'y')`.

In [ ]:
ds_mesh = ds_mesh.set_xindex(
    ['lat', 'lon'],
    xr.indexes.NDPointIndex,
    tree_adapter_cls=xoak.SklearnGeoBallTreeAdapter
)

ds_mesh

Let's create another `xarray.Dataset` of latitude / longitude points that here correspond to a trajectory on the sphere.

In [ ]:
ds_trajectory = xr.Dataset({
    'latitude': ('trajectory', np.linspace(-10, 40, 30)),
    'longitude': ('trajectory', np.linspace(-150, 150, 30))
})

ds_trajectory

We can now simply use `xarray.Dataset.sel()` to select the mesh points that are the closest to the trajectory vertices. All indexer coordinates (`latitude` and `longitude` in this example) must have the exact same dimensions (here `'trajectory'`). Indexers must be given for each coordinate used to build the index above, (here `latitude` for `lat` and `longitude` for `lon`). 

In [ ]:
ds_selection = ds_mesh.sel(
    lat=ds_trajectory.latitude,
    lon=ds_trajectory.longitude,
    method="nearest",
)

ds_selection

Let's plot the trajectory vertices (black dots) and the resulting selection (dots colored by the `field` values):

In [ ]:
fig, ax = plt.subplots()
ds_trajectory.plot.scatter(ax=ax, x='longitude', y='latitude', c='k', alpha=0.7)
ds_selection.plot.scatter(ax=ax, x='lon', y='lat', hue='field', alpha=0.9);

Xarray also supports providing coordinates with an arbitrary number of dimensions as indexers, like in the example below with vertices of another mesh on the sphere. 

In [ ]:
ds_mesh2 = xr.Dataset({
    'latitude': (('x', 'y'), np.random.uniform(-90, 90, size=(10, 10))),
    'longitude': (('x', 'y'), np.random.uniform(-180, 180, size=(10, 10)))
})

ds_selection = ds_mesh.sel(
    lat=ds_mesh2.latitude,
    lon=ds_mesh2.longitude,
    method="nearest",
)

ds_selection

In [ ]:
fig, ax = plt.subplots()
ds_mesh2.plot.scatter(ax=ax, x='longitude', y='latitude', c='k', alpha=0.7)
ds_selection.plot.scatter(ax=ax, x='lon', y='lat', hue='field', alpha=0.9);